In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18
import torch.nn as nn
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# 数据预处理操作，调整为适合SVHN数据集的归一化参数（示例参数，可根据实际情况优化）
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# 加载SVHN训练集，设置合适的参数
trainset = torchvision.datasets.SVHN(root='./data', split='train',
                                      download=True, transform=transform)
# 加载SVHN测试集
testset = torchvision.datasets.SVHN(root='./data', split='test',
                                     download=True, transform=transform)

# 划分训练集和验证集，这里采用常见的80%训练集，20%验证集划分方式
train_data, val_data = train_test_split(trainset, test_size=0.5, random_state=42)

# 创建训练集数据加载器，设置合适的参数
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64,
                                           shuffle=True, num_workers=2)
# 创建验证集数据加载器
val_loader = torch.utils.data.DataLoader(val_data, batch_size=64,
                                         shuffle=False, num_workers=2)
# 创建测试集数据加载器
test_loader = torch.utils.data.DataLoader(testset, batch_size=64,
                                          shuffle=False, num_workers=2)

# 实例化预训练的ResNet18模型
model = resnet18(pretrained=True)
# SVHN是彩色图像，输入通道数为3，所以这里不用修改conv1层的输入通道数
model.fc = nn.Linear(model.fc.in_features, 10)  # 根据SVHN的类别数量调整全连接层输出维度（假设类别数为10，可根据实际调整）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 定义损失函数，依然采用交叉熵损失函数
criterion = nn.CrossEntropyLoss()
# 定义优化器，使用Adam优化器，学习率可根据实际情况进一步调整
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


def train(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100. * correct / total
    return epoch_loss, epoch_acc


def validate(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(val_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            all_preds.extend(predicted.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())

    val_loss = running_loss / len(val_loader)
    val_acc = 100. * correct / total
    precision = precision_score(all_targets, all_preds, average='macro')
    recall = recall_score(all_targets, all_preds, average='macro')
    f1 = f1_score(all_targets, all_preds, average='macro')
    conf_matrix = confusion_matrix(all_targets, all_preds)
    return val_loss, val_acc, precision, recall, f1, conf_matrix


epochs = 5
for epoch in range(epochs):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc, precision, recall, f1, conf_matrix = validate(model, val_loader, criterion, device)
    print(f'Epoch {epoch + 1}: Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, '
          f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%, '
          f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}')
    print("Confusion Matrix:")
    print(conf_matrix)

test_loss, test_acc, precision, recall, f1, conf_matrix = validate(model, test_loader, criterion, device)
print(f'Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%, '
      f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}')
print("Confusion Matrix:")
print(conf_matrix)

Using downloaded and verified file: ./data/train_32x32.mat
Using downloaded and verified file: ./data/test_32x32.mat


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1: Train Loss: 0.6770, Train Acc: 78.84%, Val Loss: 0.4527, Val Acc: 85.80%, Precision: 0.8561, Recall: 0.8522, F1-score: 0.8484
Confusion Matrix:
[[1999   72    9   10   21    7   46   11   20  222]
 [  32 6344   21   44  291   38   16   61   42   30]
 [  13  146 4696   87   53   28    6   72   51  180]
 [   4  167   19 2964   51  352   17   15  399  239]
 [   4  108   24   11 3404   13   10    6   23   56]
 [   2   32    6   83   37 3174   21    1   21   55]
 [  40   48    7   25   55  421 2158    4   90   37]
 [   9  190   72   54   23   17    6 2490   12   21]
 [  26   58   10   25   26   83   66    2 2042  202]
 [  12   28   15   22   20   33    5   12   20 2157]]
Epoch 2: Train Loss: 0.3699, Train Acc: 89.30%, Val Loss: 0.3641, Val Acc: 89.19%, Precision: 0.9002, Recall: 0.8778, F1-score: 0.8866
Confusion Matrix:
[[2083  155   11   14   25    3   43   26    6   51]
 [  12 6681   33   30   42   10    9   90    3    9]
 [   9   90 4882   26   56   11   11  210    1   36]
 [  